## LeNet with BatchNorm

In [1]:
!nvidia-smi

Sat Mar 19 19:24:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:0B.0 Off |                    0 |
| N/A   40C    P0    44W / 300W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import paddle
import numpy as np

paddle.__version__, paddle.device.get_device()

('2.2.2', 'gpu:0')

### Data loading

In [3]:
import paddle.vision.transforms as T
from paddle.vision.datasets import FashionMNIST

# loading and normalization
transform = T.Normalize(mean=[127.5], std=[127.5])  

# constructing traning set and test set
fashionmnist_train = FashionMNIST(mode='train', transform=transform)
fashionmnist_test = FashionMNIST(mode='test', transform=transform)

In [4]:
# nums of train set and test set
len(fashionmnist_train), len(fashionmnist_test)

(60000, 10000)

In [5]:
# As for one sample, channel = 1, height, weight = 28
fashionmnist_train[0][0].shape

(1, 28, 28)

In [6]:
train_loader = paddle.io.DataLoader(fashionmnist_train, batch_size=256, shuffle=True)
test_loader = paddle.io.DataLoader(fashionmnist_test, batch_size=64, shuffle=False)

### Model

In [7]:
from paddle import nn

bn_lenet = net = nn.Sequential(
    nn.Conv2D(1, 6, kernel_size=5), nn.BatchNorm2D(6), nn.Sigmoid(),
    nn.AvgPool2D(kernel_size=2, stride=2),
    nn.Conv2D(6, 16, kernel_size=5), nn.BatchNorm2D(16), nn.Sigmoid(),
    nn.AvgPool2D(kernel_size=2, stride=2), nn.Flatten(),
    nn.Linear(256, 120), nn.BatchNorm1D(120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.BatchNorm1D(84), nn.Sigmoid(),
    nn.Linear(84, 10))
    

W0319 19:25:04.261478  2096 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0319 19:25:04.265938  2096 device_context.cc:465] device: 0, cuDNN Version: 7.6.


In [8]:
# instantiation
model = paddle.Model(bn_lenet)

# visualization of the model workflow
model.summary((-1, 1, 28, 28)) # [N C H W]

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-1       [[1, 1, 28, 28]]      [1, 6, 24, 24]          156      
 BatchNorm2D-1    [[1, 6, 24, 24]]      [1, 6, 24, 24]          24       
   Sigmoid-1      [[1, 6, 24, 24]]      [1, 6, 24, 24]           0       
  AvgPool2D-1     [[1, 6, 24, 24]]      [1, 6, 12, 12]           0       
   Conv2D-2       [[1, 6, 12, 12]]      [1, 16, 8, 8]          2,416     
 BatchNorm2D-2    [[1, 16, 8, 8]]       [1, 16, 8, 8]           64       
   Sigmoid-2      [[1, 16, 8, 8]]       [1, 16, 8, 8]            0       
  AvgPool2D-2     [[1, 16, 8, 8]]       [1, 16, 4, 4]            0       
   Flatten-1      [[1, 16, 4, 4]]          [1, 256]              0       
   Linear-1          [[1, 256]]            [1, 120]           30,840     
 BatchNorm1D-1       [[1, 120]]            [1, 120]             480      
   Sigmoid-3         [[1, 120]]     

{'total_params': 45330, 'trainable_params': 44426}

### Training

In [9]:
# optimizer and loss
model.prepare(optimizer=paddle.optimizer.Adam(parameters=model.parameters(), learning_rate=1e-3),
              loss=nn.CrossEntropyLoss(),
              metrics=paddle.metric.Accuracy())

# training
model.fit(train_loader,
        # eval_data = test_loader
        epochs=20,
        verbose=1,
        )

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/20


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:653: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


step 235/235 [==============================] - loss: 0.6304 - acc: 0.7492 - 28ms/step          
Epoch 2/20
step 235/235 [==============================] - loss: 0.3582 - acc: 0.8367 - 28ms/step          
Epoch 3/20
step 235/235 [==============================] - loss: 0.4655 - acc: 0.8541 - 27ms/step          
Epoch 4/20
step 235/235 [==============================] - loss: 0.2740 - acc: 0.8659 - 28ms/step          
Epoch 5/20
step 235/235 [==============================] - loss: 0.3584 - acc: 0.8737 - 27ms/step          
Epoch 6/20
step 235/235 [==============================] - loss: 0.3444 - acc: 0.8808 - 27ms/step          
Epoch 7/20
step 235/235 [==============================] - loss: 0.3056 - acc: 0.8843 - 28ms/step          
Epoch 8/20
step 235/235 [==============================] - loss: 0.2926 - acc: 0.8899 - 28ms/step          
Epoch 9/20
step 235/235 [==============================] - loss: 0.2758 - acc: 0.8928 - 28ms/step          
Epoch 10/20
step 235/235 [=============

In [10]:
model.evaluate(fashionmnist_test, batch_size=64, verbose=1)

Eval begin...
step 157/157 [==============================] - loss: 0.4371 - acc: 0.8902 - 8ms/step          
Eval samples: 10000


{'loss': [0.43707958], 'acc': 0.8902}